In [138]:
import cvxpy as cp
import numpy as np

## Set up constants

# Load data files
transport_cost = np.loadtxt("transportation_cost.csv", delimiter=",",skiprows=1)
transport_time = np.loadtxt("transportation_time.csv", delimiter=",",skiprows=1)
accomodations = np.loadtxt("accomodation.csv", delimiter=",",skiprows=1)
cities = np.loadtxt("cities.csv", delimiter=",",dtype=str,skiprows=1)

# Number of cities
num_cities = len(cities)

# Total budget and length of trip
budget = 1500
trip_length = 14

In [139]:
## Set up variables
M = 100000000   # Some arbitrarily large term

# Costs per day (in USD)
cost_weekend = [60, 20, 40, 25, 25, 20, 30, 30]  # Fri/Sat
cost_transition = [25, 15, 25, 20, 30, 20, 30, 25]  # Sun/Thu
cost_weekday = [25, 15, 20, 20, 20, 20, 30, 25]  # Mon/Tue/Wed

# Travel costs (matrix in USD)
travel_cost = [
    [M, 40, 65, 20, 50, 30, 50, 60],
    [40, M, 105, 40, 130, 70, 165, 130],
    [65, 105, M, 100, 60, 25, 30, 40],
    [20, 40, 100, M, 86, 20, 66, 75],
    [50, 130, 60, 86, M, 100, 80, 20],
    [30, 70, 25, 20, 100, M, 90, 130],
    [50, 165, 30, 66, 80, 90, M, 40],
    [60, 130, 40, 75, 20, 130, 40, M]
]

# Decision variables
x = cp.Variable(num_cities, integer=True)  # Total days spent in each city

y_weekday = cp.Variable(num_cities, integer=True)
y_weekend = cp.Variable(num_cities, integer=True)
y_transition = cp.Variable(num_cities, integer=True)

t = cp.Variable((num_cities, num_cities), boolean=True)  # Travel route binary variables -- traveling from city i to j

visited = cp.Variable(num_cities,boolean=True)  # Binary variable indicating whether city is visited

u = cp.Variable(num_cities,nonneg=True) # artificial variable for eliminating subtours

In [140]:
## Create objective

# Objective: Minimize total cost
total_cost = (
    sum(cost_weekend[i] * y_weekend[i] for i in range(num_cities)) +
    sum(cost_weekday[i] * y_weekday[i] for i in range(num_cities)) +
    sum(cost_transition[i] * y_transition[i] for i in range(num_cities)) +
    sum(
        travel_cost[i][j] * t[i, j] 
        for i in range(num_cities) 
        for j in range(num_cities)
    )
)

# Constraints
constraints = [
    x >= 0, # Additional nonnegativity constraints
    y_weekday >= 0,
    y_weekend >= 0,
    y_transition >= 0,
    x <= 3,  # No more than 3 days in a single city
    sum(x) == trip_length,  # Total days equal to trip duration
    sum(
        cost_weekend[i] * y_weekend[i] +
        cost_weekday[i] * y_weekday[i] +
        cost_transition[i] * y_transition[i] +
        sum(travel_cost[i][j] * t[i, j] for j in range(num_cities))
        for i in range(num_cities)
    ) <= budget # Budget constraint
]

# Total of each type of days of the week on the trip
constraints.append(sum(y_weekday) == 6)
constraints.append(sum(y_weekend) == 4)
constraints.append(sum(y_transition) == 4)

# Number of days of each type in each city
for i in range(num_cities):
    constraints.append(y_weekday[i] <= 3)
    constraints.append(y_weekend[i] <= 2)
    constraints.append(y_transition[i] <= 2)

# Individual constraints for day breakdown
for i in range(num_cities):
    constraints.append(y_weekday[i] + y_weekend[i] + y_transition[i] == x[i])

In [141]:
# Each city visited only once
for i in range(num_cities): 
    constraints.append(sum(t[i, j] for j in range(num_cities)) == 1)
    constraints.append(sum(t[j, i] for j in range(num_cities)) == 1)
 
# No subtours
for i in range(1,num_cities):
   for j in range(1,num_cities):
       constraints.append(u[i] - u[j] + num_cities*t[i,j] <= num_cities-1)

In [142]:
# If we spend no time in a city, we can't travel to and from it
for i in range(num_cities): 
    constraints.append(sum(t[i,:]) == 1000*(1-visited[i]))
    constraints.append(0 <= x[i] + 1000*visited[i])

In [143]:
# Solver
problem = cp.Problem(cp.Minimize(total_cost), constraints)
problem.solve(solver=cp.GUROBI)

# Results
print("Optimal Cost:", problem.value)
print("Days in each city:", [round(val) for val in x.value])
print("Weekdays in each city:", [round(val) for val in y_weekday.value])
print("Weekends in each city:", [round(val) for val in y_weekend.value])
print("Transits in each city:", [round(val) for val in y_transition.value])

print("Whether city was visited:", visited.value)
print("Route:")
print(t.value)

Optimal Cost: None


TypeError: 'NoneType' object is not iterable